## Classifier  approach
---
This approach assumes that quantifiers are learned as a group and that essentially each q quantifier example is a negative example for all other quantifiers q'.

The classifier is in effect a solver for which q makes the sentence "Q as are bs" most likely given an input scene s.

This enables us to use not only the quantifier quantify evaluation methods but the classifier in order to generate a teacher-student scheme.

## Imports

### my class imports

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from quants.quantifiers import *
from quants.classifiers import SingleLabelClassifier, MultiLabelClassifier, CNNClassifier

### Global imports

In [6]:
import sys
import numpy as np
import pandas as pd

### keras and TF imports

In [7]:
import keras

import tensorflow as tf

print("TensorFlow version: ", tf.__version__)

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import SimpleRNN, LSTM, Embedding, Dense, Conv1D, Input, Bidirectional, RepeatVector, Dropout, LeakyReLU, Flatten, Concatenate
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import initializers

from tensorflow.python.keras import backend as K
from keras.utils import np_utils

TensorFlow version:  2.4.0


In [11]:
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

print(tf.config.list_physical_devices(device_type='CPU'))
# print("Keras backend: ", tf.compat.v1.keras.backend.backend())
# print(tf.config.list_logical_devices())

# gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.3)
# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
# K.set_session(sess)

Python version
3.8.6 (default, Jan 27 2021, 15:42:20) 
[GCC 10.2.0]
Version info.
sys.version_info(major=3, minor=8, micro=6, releaselevel='final', serial=0)
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


### Classifier models

In [15]:
class DDNNSoftmaxClassifier(SingleLabelClassifier):
    """ deep softmax dense classifier model builder method """
    
    def build(self):
        model= Sequential()
        model.add(Dense(Quantifier.scene_len, activation="relu", name="input"))
        model.add(Dropout(0.25, name="dropout_1"))
        model.add(Dense(100, activation="relu", name="dense_2"))
        model.add(Dropout(0.25, name="dropout_2"))
        model.add(Dense(50, activation="relu", name="dense_3"))
        model.add(Dropout(0.25, name="dropout_3"))
        model.add(Dense(len(self._quantifiers), activation='softmax', name="softmax_1"))
        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.Precision(),
                                                                                  tf.keras.metrics.Recall()])
        return model

In [16]:
class DNNSoftmaxClassifier(SingleLabelClassifier):
    """ dense classifier model builder method """
    
    def build(self):
        model= Sequential()
        model.add(Dense(Quantifier.scene_len, activation="relu", name="input"))
        model.add(Dropout(0.5, name="dropout_1"))`
        model.add(Dense(len(self._quantifiers), activation='softmax', name="softmax_1"))
        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.Precision(),
                                                                                  tf.keras.metrics.Recall()])
        return model

SyntaxError: invalid syntax (<ipython-input-16-b27f3e363545>, line 7)

In [17]:
class MyClassifier(CNNClassifier, SingleLabelClassifier):
    """
    classifier class
    classifies list of quantifiers
    """

    def build(self):
        num_kernels = len(self._kernels)
        const_initializer_0 = tf.keras.initializers.Constant(0.)
        const_initializer_1 = tf.keras.initializers.Constant(1.)
        # input layer
        scene = Input(name='input', shape=(Quantifier.scene_len, len(symbols)))
        # conv
        conv = Conv1D(filters=num_kernels, kernel_size=1,
                      kernel_initializer=const_initializer_1,
                      trainable=False,
                      use_bias=False,
                      name='conv')(scene)
        # split to handle each kernel seperately
        splitters = tf.split(conv, num_kernels, axis=2, name='split')
        # flatten the result (reshapes)
        flats = [Flatten(name='flat_{i}'.format(i=i))(splitters[i])
                 for i in range(num_kernels)]
        # dropouts after convolutions
        dropouts = [Dropout(rate=0.01, name='dropout_{i}'.format(i=i))(flats[i])
                    for i in range(num_kernels)]
        # single neuron summarizers
        denses = [Dense(1,
                        kernel_initializer=const_initializer_1,
                        use_bias=False,
                        trainable=False,
                        activation='linear',
                        name='dense_{i}'.format(i=i))(dropouts[i])
                  for i in range(num_kernels)]
        # merge feature extractors
        merge = tf.concat(denses, axis=1, name='concatenate')
        # softmax layer
        dense = Dense(len(self._quantifier_names),
                        kernel_initializer=const_initializer_1,
                        use_bias=True,
                        trainable=True,
                        activation='relu', name="dense")(merge)
        softmax = Dense(len(self._quantifier_names),
                        kernel_initializer=const_initializer_0,
                        use_bias=True,
                        trainable=True,
                        activation='softmax', name="softmax")(dense)
        # inputs outputs
        model = Model(inputs=scene, outputs=softmax)
        # set weights
        conv = model.get_layer('conv')
        conv.set_weights([np.array([self._kernels]).transpose().reshape(1, len(symbols), num_kernels)])
        print(conv.get_weights())
        # compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam',
                      metrics=[
                          tf.keras.metrics.Precision()
                          , tf.keras.metrics.Recall()
                      ])
        return model

## Quantifier sets for learning

In [18]:
natural_quantifiers = [The(), Both(), No(), All(), Some(), Most()]
# natural_quantifiers = [All(), Most(), Some()]
most_quantifiers = [Most(), Not(Most())]
every_quantifiers = [All2()]
quantifiers = [Between(2, 50), All()]
monotonicity_quantifiers = [Most(), Between(20, 50)]
unnatural_quantifiers = [Between(20, 50), Between(8, 40), Between(12, 35)]

In [19]:
classifier = MyClassifier(kernels=[[1, 0, 0, 0], [1, -1, 0, 0], [0, 1, 0, 0]],
                          quantifiers=[Odd()], other=True)

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [165]:
classifier.plot()

In [166]:
classifier.learn(epochs=25, scene_num=1024, batch_size=1, max_len=500, verbose=1,
                contrastive_quantifiers=natural_quantifiers)

TRAIN
Epoch 1/25
7168/7168 [==============================] - 8s 1ms/step - loss: 0.8054 - precision_17: 0.5682 - recall_17: 0.5677
Epoch 2/25
7168/7168 [==============================] - 8s 1ms/step - loss: 0.6852 - precision_17: 0.5688 - recall_17: 0.5688
Epoch 3/25
7168/7168 [==============================] - 8s 1ms/step - loss: 0.6915 - precision_17: 0.5539 - recall_17: 0.5539
Epoch 4/25
7168/7168 [==============================] - 8s 1ms/step - loss: 0.6878 - precision_17: 0.5623 - recall_17: 0.5623
Epoch 5/25
7168/7168 [==============================] - 9s 1ms/step - loss: 0.6853 - precision_17: 0.5735 - recall_17: 0.5735
Epoch 6/25
7168/7168 [==============================] - 8s 1ms/step - loss: 0.6887 - precision_17: 0.5679 - recall_17: 0.5679
Epoch 7/25
7168/7168 [==============================] - 8s 1ms/step - loss: 0.6850 - precision_17: 0.5617 - recall_17: 0.5617
Epoch 8/25
7168/7168 [==============================] - 8s 1ms/step - loss: 0.6378 - precision_17: 0.6211 - reca

/home/doron/git/research/RESEARCH/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


32/32 [==============================] - 0s 1ms/step
Quantifier counts:  [990.  10.]
Support:  497


In [167]:
conv = classifier._model.get_layer(name="conv")
print(conv.output_shape)
print(conv.get_weights())

dense = classifier._model.get_layer(name="dense")
print(dense.output_shape)
print(dense.get_weights())

softmax = classifier._model.get_layer(name="softmax")
print(softmax.output_shape)
print(softmax.get_weights())

# for quantifier_name, weights in zip(classifier._quantifier_names, 
# # , bias 
#                                           conv.get_weights()[0].transpose(),
# #                                           conv.get_weights()[1].transpose()
#                                          ):
#     print(quantifier_name, weights)  # , bias)

# for i in range(len(classifier._kernels)):
#     dense_i = classifier._model.get_layer(name="dense_{i}".format(i=i))
#     print(dense_i.output_shape)
#     print(dense_i.get_weights())

(None, 500, 3)
[array([[[ 1.,  1.,  0.],
        [ 0., -1.,  1.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]], dtype=float32)]
(None, 2)
[array([[-3.8773236 , -3.8773236 ],
       [ 0.17213197,  0.17213197],
       [ 0.19392583,  0.19392583]], dtype=float32), array([3.7413113, 3.7413113], dtype=float32)]
(None, 2)
[array([[-0.22588426,  0.22588433],
       [-0.22588426,  0.22588433]], dtype=float32), array([ 0.17653447, -0.1765345 ], dtype=float32)]


In [127]:
a = classifier.predict(classifier.prepare_scenes(N(5).generate_scenes()))

32/32 [==============================] - 0s 1ms/step


In [128]:
np.unique(a, axis=0, return_counts=True)

(array([[0., 1.],
        [1., 0.]], dtype=float32),
 array([612, 388]))